In [1]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('/JUET/Projects/reunion-data-assignment/Problem1/ecommerce.db')

In [2]:
# 1. Retrieve the top 5 customers who have made the highest average order amounts in the last 6 months. The average order 
#    amount should be calculated for each customer, and the result should be sorted in descending order.

cursor = conn.cursor()
cursor.execute('''
    SELECT Customers.customer_id, Customers.name, AVG(Orders.total_amount) AS average_order_amount
    FROM Customers
    JOIN Orders ON Customers.customer_id = Orders.customer_id
    WHERE Orders.order_date >= DATE('now', '-6 months')
    GROUP BY Customers.customer_id, Customers.name
    ORDER BY average_order_amount DESC
    LIMIT 5;
    ''')

query = cursor.fetchall()
print(query)

[(5, 'Nathan Townsend', 747.74), (4, 'Evelyn Rivera', 714.765), (6, 'Angela Brennan', 660.82), (2, 'Jennifer Wilson', 638.7679999999999), (9, 'Madison Poole', 510.3033333333333)]


In [3]:
# 2. Retrieve the list of customer whose order value is lower this year as compared to previous year

cursor = conn.cursor()
cursor.execute('''
    SELECT Customers.customer_id, Customers.name
    FROM Customers
    JOIN Orders ON Customers.customer_id = Orders.customer_id
    WHERE strftime('%Y', Orders.order_date) = strftime('%Y', 'now')
    AND Orders.total_amount < (
        SELECT SUM(total_amount)
        FROM Orders
        WHERE strftime('%Y', order_date) = strftime('%Y', 'now', '-1 year')
        AND Customers.customer_id = Orders.customer_id
    );
    ''')

query = cursor.fetchall()
print(query)

[(4, 'Evelyn Rivera'), (10, 'Alexandra Avery'), (2, 'Jennifer Wilson'), (9, 'Madison Poole'), (10, 'Alexandra Avery'), (10, 'Alexandra Avery'), (2, 'Jennifer Wilson'), (6, 'Angela Brennan'), (5, 'Nathan Townsend'), (2, 'Jennifer Wilson'), (3, 'Thomas Petersen'), (7, 'Ashley Hughes'), (1, 'Sarah Smith'), (4, 'Evelyn Rivera'), (6, 'Angela Brennan'), (10, 'Alexandra Avery'), (5, 'Nathan Townsend'), (9, 'Madison Poole'), (3, 'Thomas Petersen'), (6, 'Angela Brennan'), (8, 'Ricardo Kaufman'), (2, 'Jennifer Wilson'), (9, 'Madison Poole'), (2, 'Jennifer Wilson')]


In [4]:
# 3. Create a table showing cumulative purchase by a particular customer

cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE CumulativePurchase AS
    SELECT Orders.order_id, Orders.customer_id, Orders.order_date, SUM(Orders.total_amount) OVER (PARTITION BY Orders.customer_id ORDER BY Orders.order_date) AS cumulative_purchase
    FROM Orders;
    ''')

cursor.execute("SELECT * FROM CumulativePurchase;")
query = cursor.fetchall()
print(query)

OperationalError: table CumulativePurchase already exists

In [19]:
# 4. Retrieve the list of top 5 selling products. Further bifurcate the sales by product variants

cursor = conn.cursor()
cursor.execute('''
    SELECT Products.name, SUM(OrderItems.quantity) AS total_quantity
    FROM Products
    JOIN OrderItems ON Products.product_id = OrderItems.product_id
    GROUP BY Products.name
    ORDER BY total_quantity DESC
    LIMIT 5;
    ''')

query = cursor.fetchall()
print(query)

[('iPhone', 362), ('MacBook', 353), ('iPod', 224), ('Apple TV', 203), ('Beats', 151)]


In [20]:
cursor = conn.cursor()
cursor.execute('''
    SELECT Products.name, Variants.name, SUM(OrderItems.quantity) AS total_quantity
    FROM Products
    JOIN OrderItems ON Products.product_id = OrderItems.product_id
    JOIN Products AS Variants ON Products.product_id = Variants.parent_product_id
    GROUP BY Products.name, Variants.name
    ORDER BY total_quantity DESC;
    ''')

query = cursor.fetchall()
print(query)

OperationalError: no such column: Variants.parent_product_id

In [14]:
conn.close()